In [3]:
import os
import pandas as pd
import requests
import shutil

for csv_file in os.listdir('data'):
    if csv_file.endswith('.csv'):
        df = pd.read_csv(os.path.join('data', csv_file))
        
        for idx, row in df.iterrows():
            if idx >= 1000:
                break
            image_url = row['product_url']
            file_name = row['bottom_category']
            image_name = f"{file_name}_{idx}.jpg"
            image_path = os.path.join('image', image_name)
            
            try:
                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(image_path, 'wb') as out_file:
                        shutil.copyfileobj(response.raw, out_file)
                    print(f"이미지가 성공적으로 저장 완료 {image_path}")
                else:
                    print(f"이미지를 다운로드하는데 실패 {image_url}")
                    continue
            except Exception as e:
                print(f"이미지를 다운로드하는 도중 예외가 발생 {e}")
                continue
            finally:
                del response



이미지가 성공적으로 저장 완료 image\가디건_0.jpg
이미지가 성공적으로 저장 완료 image\가디건_1.jpg
이미지가 성공적으로 저장 완료 image\가디건_2.jpg
이미지가 성공적으로 저장 완료 image\가디건_3.jpg
이미지가 성공적으로 저장 완료 image\가디건_4.jpg
이미지가 성공적으로 저장 완료 image\가디건_5.jpg
이미지가 성공적으로 저장 완료 image\가디건_6.jpg
이미지가 성공적으로 저장 완료 image\가디건_7.jpg
이미지를 다운로드하는데 실패 https://ccimg.hellomarket.com/images/2018/item/08/26/00/2803_2830222_1.jpg?size=s4
이미지를 다운로드하는데 실패 https://ccimg.hellomarket.com/images/2019/item/05/12/18/2417_2830222_2.jpg?size=s4
이미지가 성공적으로 저장 완료 image\가디건_10.jpg
이미지가 성공적으로 저장 완료 image\가디건_11.jpg
이미지가 성공적으로 저장 완료 image\가디건_12.jpg
이미지가 성공적으로 저장 완료 image\가디건_13.jpg
이미지가 성공적으로 저장 완료 image\가디건_14.jpg
이미지를 다운로드하는데 실패 https://ccimg.hellomarket.com/images/2019/item/01/10/20/0604_3212847_3.jpg?size=s4
이미지가 성공적으로 저장 완료 image\가디건_16.jpg
이미지가 성공적으로 저장 완료 image\가디건_17.jpg
이미지가 성공적으로 저장 완료 image\가디건_18.jpg
이미지가 성공적으로 저장 완료 image\가디건_19.jpg
이미지가 성공적으로 저장 완료 image\가디건_20.jpg
이미지가 성공적으로 저장 완료 image\가디건_21.jpg
이미지가 성공적으로 저장 완료 image\가디건_22.jpg
이미지가 성공적으로 저장 완료 image\가디건_23.j

NameError: name 'response' is not defined

In [ ]:
import requests
import shutil

def download_image(url, file_path):
    # URL로부터 이미지를 요청합니다.
    response = requests.get(url, stream=True)
    
    if response.status_code == 200:
        with open(file_path, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        print("이미지가 성공적으로 저장되었습니다.")
    else:
        print("이미지를 다운로드하는데 실패했습니다.")
    # 메모리 해제
    del response

# 사용 예제
image_url = "https://example.com/image.jpg"  # 이미지 URL을 입력하세요.
save_path = "saved_image.jpg"  # 저장할 파일 경로를 입력하세요.
download_image(image_url, save_path)

In [ ]:
# 로그인
login_url = f'http://{ip}/api/accounts/login/'
data = {
    'email': 'qortnwjd58@naver.com',
    'password': 'test123456'
}
res = requests.post(login_url, data=data)
print(res.status_code)
ac_token = res.json()['access']

In [ ]:
# 파일 (최대 5개까지만 가능함.)
upload_url = f"http://{ip}/api/images/upload/product/"
headers = {
    'Authorization': f'Bearer {ac_token}',
}
file_path1 = 'docker.png'
files = [
    ('images', open(file_path1, 'rb')),
]
response = requests.post(upload_url, headers=headers, files=files)


if response.status_code != 201:
    raise Exception("Image upload failed")
image_urls = [img['image_url'] for img in response.json()['uploaded_images']]

# 게시글 생성
article_url = f"http://{ip}/api/articles/"


data = {
    "title": "테스트2",
    "content": "테스트2",
    "product": {
        "option": {"size": "XS", "color": "어두운 빨강"},
        "category": {"top_category": "상의", "bottom_category": "니트"},
        "price": 5000,
        "product_type": 1,
        'product_status': '사용감 있는 새상품',
        "image_urls": image_urls  # image_urls를 product 안에 포함
    }
}
response = requests.post(article_url, headers=headers, json=data)
response.json()